### Imports-uvoz potrebnih modula za projekt

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
from dataclasses import dataclass, asdict

In [2]:
#Jedan pokušaj ostvarenja ideje, kako odmah stvoriti prazne liste sa predodređenim data type po mojoj volji
 #@dataclass
 #class Car:
    #naslov: str
    #godiste: int
    #kilometri: str
    #snagakW: int
    #lokacija: str
    #cijena_kn: int
    #cijena_Eur: int 

### HTTP Request-zahtjev

#### spremanje poveznice/linka, unutar varijable 'webstranica'

In [3]:
webstranica = "https://www.index.hr/oglasi/osobni-automobili/gid/27?pojamZup=-2&tipoglasa=1&grad=0&naselje=0&cijenaod=0&cijenado=46394055&num=200&elementsNum=50&sortby=1"

### Get Request
#### Spremanje unutar varijable  'povrat' web stranice uz pomoć zahtjeva .get kroz requests modul

In [4]:
povrat = requests.get(webstranica) 

### Status Code
#### Provjera statusnog koda-važno nam je radi eventualne prilagodbe ako ima problema 
#### sa zahtjevom HTTP tipa 404 ili 500 itd što spada pod error

In [5]:
povrat.status_code

200

### Soup Objekt
#### Kreiranje 'soup' objekta da bi dobili HTML elemente prertodno kreirane varijable 'povrata' ciljane web stranice

In [6]:
soup = BeautifulSoup(povrat.content, 'html.parser')   # za kreiranje objekta tribam dva argumenta, sadržaj povrata i 'html raščlanjivač'

In [7]:
soup

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="1697489853797759" property="fb:app_id"/>
<title>Rabljena vozila na prodaju | Auto-moto oglasnik</title>
<meta content="Rabljena vozila na prodaju | Auto-moto oglasnik" property="og:title"/>
<meta content="https://www.index.hr/oglasi/osobni-automobili/gid/27?pojamZup=-2&amp;tipoglasa=1&amp;grad=0&amp;naselje=0&amp;cijenaod=0&amp;cijenado=46394055&amp;num=200&amp;elementsNum=50&amp;sortby=1" property="og:url"/>
<meta content="Pitate se gdje kupiti auto? Ovo je najveća baza rabljenih automobila u Hrvatskoj! Pretražite oglase, saznajte cijenu i pogledajte slike. – Index Oglasi." name="description"/>
<meta content="www.index.hr/oglasi" property="og:site_name"/>
<link href="https://www.index.hr/oglasi/osobni-automobili/gid/27" rel="canonical"/>
<meta content="https://www.index.hr/oglasi/img/IndexOglasiLogo.jpg" property="og:image"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=3.0, minimum-scale=1.0"

### Rezultat - povrat

In [8]:
rezultat = soup.find_all('div', {'class': 'OglasiRezHolder'})   #  kreiranj varijable radi spremanja rezultata svakog automobila-objekt lista

In [9]:
len(rezultat)  # provjera koliko oglasda imamo unutar liste- imam na umu da unutar 'rezulatat' ima i oglasa

56

In [10]:
rezultat[0]     #  provjera HTML sardžaja 'rezultat' ,  iz liste sa pozicije prve a ona je na '0' 

<div class="OglasiRezHolder">
<a class="result" href="https://www.index.hr/oglasi/bmw-318i-e90/oid/3681316">
<figure class="result_photo" data-x="2">
<img alt="BMW  318i  e90" src="https://www.index.hr/oglasi/userdocsimages/oglas/_2022//10//9/3681316/ike6-091020221931165516.jpg?preset=oglas-slika-view"/>
</figure>
<span class="result_body">
<span class="head">
<span class="title px18">
BMW  318i  e90
<span class="addFavorit icon-heart tooltip_bottom" title="Spremi oglas">
</span>
</span>
<span class="lead" data-fff="bb">BMW 318i e90</span>
<ul class="tags hide-on-small-only"><li>
Godište
 : 
2007
</li>
<li>
km
 : 
190.000
</li>
<li>Starost : Rabljeno</li>
<li>
kW
 : 
95
</li>
</ul>
</span>
<span class="foot">
<ul class="info">
<li class="icon-marker">Grad Zagreb</li>
<li class="icon-time">Objava  09.10.2022.</li>
</ul>
<span class="price"><span>6.300 €</span> ~ 47.339 kn</span></span>
</span>
</a>
</div>

In [11]:
cijena_euri = rezultat[0].find('span', {'class':'price'}).span.text  # izdvajanje texta za cijenu u eurima
cijena_euri

'6.300 €'

In [12]:
cijena_kune = rezultat[0].find('span', {'class':'price'}).span.next_sibling   # izdvajanje texta za cijenu u kunama
cijena_kune

' ~ 47.339 kn'

### Izdvajanje/ciljanje - željenih informacija

In [13]:
# Naslov 
# Godište
# Kilometraža
# Snaga
# Lokacija
# Cijena

### Naslov
#### Traženje naslova oglasa unutar rezultata-prvi oglas, pomoću .find i atributa, i čišćenje texta samog uz .get i .strip funkcije

In [14]:
rezultat[0].find('span', {'class':'title px18'}).get_text().strip()

'BMW  318i  e90'

### Godište
##### Ovdje mi je sve bilo baš izazovno, jer unutar 'ul' (unordered) liste imam 'li' (listu nekoliko stavki), pa svaka stavka je razdvojena ':' delimiterom tako da baš sam se namučio u potrzi riješenja, i evo jedno od koje sam skonato i primjenio-'Godište' ,'Kilometraža', 'Snaga'

In [15]:
#godiste = rezultat[0].find('ul')
#for li in godiste.find_all("li"):
#    print(li.text, end=" ")

In [16]:
lista_ul = rezultat[0].find('ul', {'class':'tags hide-on-small-only'})
lista_li = lista_ul.find_all('li')
for stavka in lista_li:
    stavkaTXT = stavka.get_text()
    #print(stavka)
    if 'Godište' in stavkaTXT:
        godiste = stavkaTXT.split()[-1]
print(godiste)        

2007


### Kilometraža

In [17]:
lista_ul = rezultat[0].find('ul', {'class':'tags hide-on-small-only'})
lista_li = lista_ul.find_all('li')
for stavka in lista_li:
    stavkaTXT = stavka.get_text()
    #print(stavka)
    if 'km' in stavkaTXT:
        kilom = stavkaTXT.split()[-1]
        kilometri = kilom.replace('.', '')
print(kilometri)        

190000


### Snaga

In [18]:
lista_ul = rezultat[0].find('ul', {'class':'tags hide-on-small-only'})
lista_li = lista_ul.find_all('li')
for stavka in lista_li:
    stavkaTXT = stavka.get_text()
    #print(stavka)
    if 'kW' in stavkaTXT:
        snagaKW = stavkaTXT.split()[-1]
print(snagaKW) 

95


### Lokacija

In [19]:
rezultat[0].find('li', {'class':'icon-marker'}).get_text()

'Grad Zagreb'

### Cijena

In [20]:
cijena_eu = rezultat[0].find('span', {'class':'price'}).span.text
cijena_euri = cijena_eu.replace('.', '').strip(' €')
cijena_euri

'6300'

In [21]:
cijena_kn = rezultat[0].find('span', {'class':'price'}).span.next_sibling
cijena_kune = cijena_kn.replace('.', '').strip(' ~').strip(' kn')
cijena_kune

'47339'

### Stavljnje svega unutar For-Loop petlje

In [22]:
# Kreiranje praznih lista svih ciljanih informacija koje ću popunjavati
# 'try' i 'except' blok mi pomaže u slučaju reklama kojih ima na svkoj stranici cca 6 komada

naslov = []
godiste = []
kilometri = []
snagakW = []
lokacija = []
cijena_Kn = []
cijena_Eur = []


for oglas in rezultat:
    
    # naslov
    try:
        naslov.append(oglas.find('span', {'class':'title px18'}).get_text().strip())
    except:
        naslov.append('n/a')
        
        
    # godiste
    try:
        lista_ul = oglas.find('ul', {'class':'tags hide-on-small-only'})
        lista_li = lista_ul.find_all('li')
        for stavka in lista_li:
            stavkaTXT = stavka.get_text()
            if 'Godište' in stavkaTXT:
                god = stavkaTXT.split()[-1]
        godiste.append(int(god))
    except:
        godiste.append('n/a')    
    
    
    # kilometri
    try:
        lista_ul = oglas.find('ul', {'class':'tags hide-on-small-only'})
        lista_li = lista_ul.find_all('li')
        for stavka in lista_li:
            stavkaTXT = stavka.get_text()
            if 'km' in stavkaTXT:
                kilom = stavkaTXT.split()[-1]
                kilomet = kilom.replace('.', '')
        kilometri.append(str(kilomet))       
    except:
        kilometri.append('n/a')    
        
    
    # snaga
    try:
        lista_ul = oglas.find('ul', {'class':'tags hide-on-small-only'})
        lista_li = lista_ul.find_all('li')
        for stavka in lista_li:
            stavkaTXT = stavka.get_text()
            if 'kW' in stavkaTXT:
                power = stavkaTXT.split()[-1]
        snagakW.append(int(power))       
    except:
        snagakW.append('n/a')    
        
    
    # lokacija
    try:
        lokacija.append(oglas.find('li', {'class':'icon-marker'}).get_text())
    except:
        lokacija.append('n/a')
    
        
    # cijena Euri
    try:
        cijena_eu = oglas.find('span', {'class':'price'}).span.get_text()
        cijena_euri = cijena_eu.replace('.', '').strip(' €')
        cijena_Eur.append(int(cijena_euri))
    except:
        cijena_Eur.append('n/a')
    
    # cijena Kune
    try:
        cijena_kn = oglas.find('span', {'class':'price'}).span.next_sibling
        cijena_kune = cijena_kn.replace('.', '').strip(' ~').strip(' kn')
        cijena_Kn.append(int(cijena_kune))
    except:
        cijena_Kn.append('n/a')

In [23]:
cijena_Kn

[47339,
 'n/a',
 53351,
 24044,
 24789,
 4000,
 9017,
 16531,
 41063,
 22918,
 3000,
 'n/a',
 44334,
 40500,
 751,
 30138,
 139012,
 1503,
 32311,
 105199,
 2500,
 63871,
 'n/a',
 180075,
 176576,
 15028,
 179589,
 30138,
 52742,
 65550,
 41440,
 'n/a',
 150690,
 'n/a',
 64043,
 128087,
 25617,
 45000,
 78899,
 93927,
 11302,
 20343,
 9017,
 54248,
 'n/a',
 89661,
 52592,
 51089,
 4500,
 26300,
 92674,
 751,
 55002,
 31560,
 123984,
 'n/a']

### Kreiranje 'pandas' podatkovnog oblika

In [24]:
index_HR = pd.DataFrame({'Naslov':naslov, 'Godište':godiste, 'Kilometraža':kilometri,'Snaga-kW':snagakW, 
                         'Lokacija':lokacija, 'Cijena-Eur':cijena_Eur, 'Cijena-kn':cijena_Kn})

In [25]:
index_HR

,Naslov,Godište,Kilometraža,Snaga-kW,Lokacija,Cijena-Eur,Cijena-kn
0,BMW 318i e90,2007,190000,95,Grad Zagreb,6300,47339
1,n/a,n/a,n/a,n/a,n/a,n/a,n/a
2,Opel Insignia 2.0 CDTI Automatic,2011,194000,96,Zadarska,7100,53351
3,Ford Focus Karavan Ford Focus 1.8 tdci Trend,2006,312000,85,Zagrebačka,3200,24044
4,"Ford Fusion 1.4 Tdci KLIMA, Reg. 11/22",2007,270000,50,Sisačko-moslavačka,3299,24789
5,Mercedes-Benz A-klasa 170,2000,270000,50,Splitsko-dalmatinska,532,4000
6,Hyundai Elantra 1.6 GL,2001,123000,79,Grad Zagreb,1200,9017
7,Hyundai Getz Hyundai getz,2005,123000,79,n/a,2200,16531
8,BMW serija 1 118d***REG 03.06.2023***,2007,243000,105,Osječko-baranjska,5450,41063
9,Ford Focus 1.4 benzin,2008,238000,55,Brodsko-posavska,3050,22918


#### Data Cleaning

In [26]:
# index_HR['Cijena-kn'] = index_HR['Cijena-kn'].apply(lambda x:x.strip(' kn').strip(' ~'))  # uklanjanje simbola koji mi ne triba

In [27]:
# index_HR['Cijena-Eur'] = index_HR['Cijena-Eur'].apply(lambda x:x.strip(' €'))

In [28]:
# index_HR

### Output in Excel

In [29]:
index_HR.to_excel('jedna_str_Index-HR.xlsx', index=False)

### Part 2 - Pagination 

In [30]:
naslov = []
godiste = []
kilometri = []
snagakW = []
lokacija = []
cijena_Kn = []
cijena_Eur = []

for i in range(1,11):
    
    # webstranica unutar varijable 'webstranica'
    webstranica = "https://www.index.hr/oglasi/osobni-automobili/gid/27?pojamZup=-2&tipoglasa=1&grad=0&naselje=0&cijenaod=0&cijenado=46394055&num=200&elementsNum=50&sortby= + str(i)"
    
    # zahtjev prema sajtu
    povrat = requests.get(webstranica) 
    
    # soup objekt
    soup = BeautifulSoup(povrat.content, 'html.parser')
    
    # povrat-rezultat
    rezultat = soup.find_all('div', {'class': 'OglasiRezHolder'})
    
    
    for oglas in rezultat:
    
        # naslov
        try:
            naslov.append(oglas.find('span', {'class':'title px18'}).get_text().strip())
        except:
            naslov.append('NaN')


        # godiste
        try:
            lista_ul = oglas.find('ul', {'class':'tags hide-on-small-only'})
            lista_li = lista_ul.find_all('li')
            for stavka in lista_li:
                stavkaTXT = stavka.get_text()
                if 'Godište' in stavkaTXT:
                    god = stavkaTXT.split()[-1]
            godiste.append(int(god))       
        except:
            godiste.append('NaN')    


        # kilometri
        try:
            lista_ul = oglas.find('ul', {'class':'tags hide-on-small-only'})
            lista_li = lista_ul.find_all('li')
            for stavka in lista_li:
                stavkaTXT = stavka.get_text()
                if 'km' in stavkaTXT:
                    kilom = stavkaTXT.split()[-1]
                    kilomet = kilom.replace('.', '')
            kilometri.append(int(kilomet))       
        except:
            kilometri.append('NaN')    


        # snaga
        try:
            lista_ul = oglas.find('ul', {'class':'tags hide-on-small-only'})
            lista_li = lista_ul.find_all('li')
            for stavka in lista_li:
                stavkaTXT = stavka.get_text()
                if 'kW' in stavkaTXT:
                    power = stavkaTXT.split()[-1]
            snagakW.append(int(power))       
        except:
            snagakW.append('NaN')    


        # lokacija
        try:
            lokacija.append(oglas.find('li', {'class':'icon-marker'}).get_text())
        except:
            lokacija.append('NaN')


        # cijena Euri
        try:
            cijena_eu = oglas.find('span', {'class':'price'}).span.get_text()
            cijena_euri = cijena_eu.replace('.', '').strip(' €')
            cijena_Eur.append(int(cijena_euri))
        except:
            cijena_Eur.append('NaN')

        # cijena Kune
        try:
            cijena_kn = oglas.find('span', {'class':'price'}).span.next_sibling
            cijena_kune = cijena_kn.replace('.', '').strip(' ~').strip(' kn')
            cijena_Kn.append(int(cijena_kune))
        except:
            cijena_Kn.append('NaN')
        

In [31]:
index_HR = pd.DataFrame({'Naslov':naslov, 'Godište':godiste, 'Kilometraža':kilometri,'Snaga-kW':snagakW, 
                         'Lokacija':lokacija, 'Cijena-Eur':cijena_Eur, 'Cijena-Kn':cijena_Kn})

In [32]:
index_HR

,Naslov,Godište,Kilometraža,Snaga-kW,Lokacija,Cijena-Eur,Cijena-Kn
0,BMW 318i e90,2007,190000,95,Grad Zagreb,6300,47339
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Opel Insignia 2.0 CDTI Automatic,2011,194000,96,Zadarska,7100,53351
3,Ford Focus Karavan Ford Focus 1.8 tdci Trend,2006,312000,85,Zagrebačka,3200,24044
4,"Ford Fusion 1.4 Tdci KLIMA, Reg. 11/22",2007,270000,50,Sisačko-moslavačka,3299,24789
...,...,...,...,...,...,...,...
555,Navi .radio.itd,2022,220000,100,Zagrebačka,100,751
556,VW Polo 1.6 tdi,2011,280000,77,Splitsko-dalmatinska,7300,55002
557,Renault Espace,2008,250000,124,Grad Zagreb,4200,31560
558,VW Golf VII GTD,2014,102000,135,Grad Zagreb,16500,123984


### Data clenanig

In [33]:
#index_HR.loc[index_HR['Naslov'] != 'NaN'].reset_index()  # locira i izdvaja sve redove osim sa vrijednosti 'NaN'

#index_HR[index_HR.Naslov != 'NaN'].reset_index()  # locira i izdvaja sve redove osim sa vrijednosti 'NaN' Obavezan je RESET index poretka

index_HR.drop(index_HR[index_HR['Naslov'] == 'NaN'].index, inplace = True) # Još jedna metoda izmjene i trajne promjene
index_HR.reset_index()

,index,Naslov,Godište,Kilometraža,Snaga-kW,Lokacija,Cijena-Eur,Cijena-Kn
0,0,BMW 318i e90,2007,190000,95,Grad Zagreb,6300,47339
1,2,Opel Insignia 2.0 CDTI Automatic,2011,194000,96,Zadarska,7100,53351
2,3,Ford Focus Karavan Ford Focus 1.8 tdci Trend,2006,312000,85,Zagrebačka,3200,24044
3,4,"Ford Fusion 1.4 Tdci KLIMA, Reg. 11/22",2007,270000,50,Sisačko-moslavačka,3299,24789
4,5,Mercedes-Benz A-klasa 170,2000,270000,50,Splitsko-dalmatinska,532,4000
...,...,...,...,...,...,...,...,...
495,554,Audi A1 1.6 tdi S-line,2012,214000,77,Istarska,12300,92674
496,555,Navi .radio.itd,2022,220000,100,Zagrebačka,100,751
497,556,VW Polo 1.6 tdi,2011,280000,77,Splitsko-dalmatinska,7300,55002
498,557,Renault Espace,2008,250000,124,Grad Zagreb,4200,31560


In [34]:
# index_HR['Cijena-Kn'] = index_HR['Cijena-Kn'].apply(lambda x:x.strip(' kn').strip(' ~'))  # uklanjanje simbola koji mi ne triba
# index_HR['Cijena-Eur'] = index_HR['Cijena-Eur'].apply(lambda x:x.strip(' €'))

In [36]:
index_HR.to_excel('01deset_uredjeno_str_Index-HR.xlsx', index=False) # Izvoz DB u excel 